In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import shutil
from functions import *

In [9]:
data_rx = pd.read_excel('precos_genericos_alterado.xlsx')
data_rx.head()

,ean,produto,laboratorio,codshopfarma,preco_compra,margem,preco_shopfarma,preco_preco_popular,preco_saojoao,preco_medio,preco_minimo,neg,preco_min_medio,desconto_pmedio,desconto_pminimo,desconto_pminmedio,desconto_cliente,desconto_max
0,7899620912731,ACEBROFILINA XPE AD 120ML,GLOBO,140253,20.29,33.021193,26.99,NaN,NaN,26.99,26.99,6.70,26.990,0.000000,0.000000,0.000000,10.0,20.0
1,7896523209111,ACEBROFILINA XPE AD 120ML,CIMED,113267,8.89,169.853768,23.99,20.65,NaN,22.32,20.65,11.76,21.485,6.961234,13.922468,10.441851,10.0,20.0
2,7896112191537,ACEBROFILINA XPE AD 120ML,TEUTO,110884,10.02,179.441118,28.00,23.26,NaN,25.63,23.26,13.24,24.445,8.464286,16.928571,12.696429,10.0,20.0
3,7895296271035,ACEBROFILINA XPE AD 120ML,NOVA QUIMICA,16545,14.72,121.942935,32.67,NaN,NaN,32.67,32.67,17.95,32.670,0.000000,0.000000,0.000000,10.0,20.0
4,7896714214146,ACEBROFILINA XPE AD 120ML,NEO QUIMICA,8731,7.99,381.852315,38.50,NaN,NaN,38.50,38.50,30.51,38.500,0.000000,0.000000,0.000000,10.0,20.0


In [10]:
def inclui_preco_cliente(df, filial=None):
    driver = webdriver.Chrome()#options=options)
    driver.get(f"http://179.184.16.200:4647/sgfpod1/Cad_0218.pod")
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(login_trier)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(password_trier)
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(1)
    if filial == None:
            pass
    else:
        driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(filial)
        driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
        sleep(1)
    driver.find_element(By.NAME, 'codTipoClienteEntrada').click()
    sleep(3)
    driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys('3')
    driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys(Keys.RETURN)
    driver.find_element(By.ID, 'produtoLabel').click()
    sleep(1)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            driver.find_element(By.NAME, 'codTipoEntrada').send_keys(row['codshopfarma'])
            driver.find_element(By.NAME, 'codTipoEntrada').send_keys(Keys.RETURN)
            sleep(1)
            desconto_cliente = str(int(row['desconto_cliente'])).replace('.',',')
            driver.find_element(By.NAME, 'prc_descto').send_keys(desconto_cliente)
            desconto_max_cliente = str(int(row['desconto_max'])).replace('.',',')
            driver.find_element(By.NAME, 'prc_desctomaximo').send_keys(desconto_max_cliente)
            sleep(1)
            driver.find_element(By.NAME, 'salvar').click()
            sleep(3)
        except:
            try:
                driver.find_element(By.ID, 'bt_yes').click()
                sleep(3)
            except:
                driver.refresh()
                sleep(3)
                if filial == None:
                    driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
                else:
                    driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(filial)
                    driver.find_element(By.NAME, 'cod_filialEntrada').send_keys(Keys.RETURN)
                    sleep(1)
                driver.find_element(By.NAME, 'codTipoClienteEntrada').click()
                sleep(3)
                driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys('3')
                driver.find_element(By.NAME, 'codTipoClienteEntrada').send_keys(Keys.RETURN)
                
                sleep(2)
    sleep(10)

,ean,produto,laboratorio,codshopfarma,preco_compra,margem,preco_shopfarma,preco_preco_popular,preco_saojoao,preco_medio,preco_minimo,neg,preco_min_medio,desconto_pmedio,desconto_pminimo,desconto_pminmedio,desconto_cliente,desconto_max
1282,7896004720937,BETAM+AC SALICIL SOL TOP 30ML,GERMED,113550,8.30,286.987952,32.12,18.19,NaN,25.155000,18.19,9.89,21.672500,21.684309,43.368618,32.526463,21.684309,43.368618
1283,7899547503913,ALOPURINOL 300MG 30CP,PRATI DONADUZZI,14319,8.56,362.383178,39.58,19.21,34.19,30.993333,19.21,10.65,25.101667,21.694458,51.465387,36.579923,21.694458,51.465387
1284,7898216361571,NEOMICINA+BACIT POM DERM 10G,PHARLAB,38618,3.94,281.218274,15.02,8.50,NaN,11.760000,8.50,4.56,10.130000,21.704394,43.408788,32.556591,21.704394,43.408788
1285,7896004727882,BROMOPRIDA 10MG 20CAP,EMS,112856,7.02,370.655271,33.04,18.35,26.19,25.860000,18.35,11.33,22.105000,21.731235,44.461259,33.096247,21.731235,44.461259
1286,7896004709291,DEFLAZACORTE 30MG 10CP,EMS,5456,61.42,134.646695,144.12,89.38,104.89,112.796667,89.38,27.96,101.088333,21.734203,37.982237,29.858220,21.734203,37.982237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,7896004703794,TERBINAFINA 250MG 14CP,EMS,4983,23.63,510.537452,144.27,26.58,NaN,85.425000,26.58,2.95,56.002500,40.788106,81.576211,61.182158,40.000000,70.000000
2196,7896422513951,VENLAFAXINA 75MG 30CAP L.C,MEDLEY GENERICOS,16539,24.17,319.528341,101.40,48.95,26.63,58.993333,26.63,2.46,42.811667,41.821170,73.737673,57.779421,40.000000,70.000000
2197,7896004746388,VENLAFAXINA 75MG 30CAP L.P,EMS,55784,21.56,419.480519,112.00,28.11,60.99,67.033333,28.11,6.55,47.571667,40.148810,74.901786,57.525298,40.000000,70.000000
2198,7896004725000,ZOLPIDEM 10MG 20CP REV,EMS,113758,8.76,646.118721,65.36,21.15,27.69,38.066667,21.15,12.39,29.608333,41.758466,67.640759,54.699612,40.000000,67.640759


In [14]:
inclui_preco_cliente(data_rx[1282:])

100%|██████████| 918/918 [1:27:11<00:00,  5.70s/it]
